In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from pandas import Series, DataFrame
from selenium.webdriver.common.keys import Keys
import time
import re

In [2]:
def data_to_csv(dict_n):

    git_address = r'C://Users//Jeong/datatest/'
    for key, an_attracrion_dict in dict_n.items():
        an_attracrion_dict.to_csv(git_address + key + '.csv', mode = 'w')

In [19]:
def reviews(sample_size):
    
    reviews = []
    columns = ['title','text', 'contributors', 'helpful', 'hometown']
        
        
    #리뷰 크기를 텍스트로 찾아와, 콤마를 제거하여, 정수형으로 저장.
    review_size_text = driver.find_element_by_class_name('attractions-community-content-TabBarContent__tabCount--2hTdj').text
    review_size = int(re.sub(',','', str(review_size_text)))
    
    
    n = 5
    count = 0
    while n < sample_size:
        try:
            foot = driver.find_element_by_class_name('location-review-review-list-parts-ReviewScrollController__container--3XtvE')
            show_more = foot.find_element_by_tag_name('button')  
            driver.execute_script("arguments[0].click();", show_more)
            n += 5
        except:
            if count > 3:
                break
            count +=1
            time.sleep(2)
            
    review_cards = driver.find_elements_by_class_name('location-review-card-Card__ui_card--2Mri0')

    for review_card in review_cards:
        try:
            read_more = review_card.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
            read_more.click()
        except:
            print('error')
            continue
    number_count = 0
    # 드라이버 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review_elements = soup.find_all(class_ = 'location-review-card-Card__ui_card--2Mri0 location-review-card-Card__card--o3LVm location-review-card-Card__section--NiAcw')

    
    
    
    for review_element in review_elements:
        title = review_element.find('a', class_='location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT').find('span').find('span').text,
        try:
            text = review_element.find('q', class_='location-review-review-list-parts-ExpandableReview__reviewText--gOmRC').find('span').text,
        except AttributeError:
            print('no text')
            text = float('nan')
        try:
            contribution = review_element.find_all(class_='social-member-MemberHeaderStats__bold--3z3qh')[0].text
            helpful = review_element.find_all(class_='social-member-MemberHeaderStats__bold--3z3qh')[1].text
        except IndexError:
            helpful=float('nan')
            
        raw_hometown = review_element.find(class_='default social-member-common-MemberHometown__hometown--3kM9S small')
        try:
            hometown = raw_hometown.text
        except AttributeError:
            print('None')
            hometown = float('nan')
        review = [
            title,
            text,
            contribution,
            helpful,
            hometown
        ]
        reviews.append(review)

    return DataFrame(reviews, columns = columns)

In [16]:
print(contribution)

NameError: name 'contribution' is not defined

In [4]:
def load_reviews(link):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 어떤 즐길거리 번호 int, 이름 string 예: "Gyeongbokgung Palace"
    출력 : 어트랙션의 리뷰 모음. data.frame
    '''
    #새 창을 열고 링크로 간다.
    # 슬립...매우매우중요이거없으면무조건에러남.이거하나로3시간머리싸맸음.
    
    
    time.sleep(5)
    original_window = driver.current_window_handle
    driver.execute_script("arguments[0].click();", link)
    
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            break
    
    #즐길거리 별로 샘플사이즈를 어떻게 정할 것인지?    
    result = reviews(300)
    
    if result['text'].isnull().sum() != 0:
        print(result['text'].isnull().sum())
        result.head()

    #창을 닫고 리스트 창으로 돌아간다
    driver.close();
    driver.switch_to.window(original_window)
    driver.current_window_handle;
    
    return result
    #result.head()

In [5]:
def load_page(start, end):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 크롤링할 페이지 내 시작 번호와 끝 번호
    출력 : 페이지의 리뷰들을 csv파일로 출력
    '''

    #모든 여행지의 리스트를 만든다
    is_first = True
    print("I am in the load_page")
    if driver.current_url == 'https://www.tripadvisor.com/Attractions-g294197-Activities-Seoul.html':
        main_page = driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__container--3eHZU')
        attraction_elements = main_page.find_elements_by_class_name("attractions-attraction-overview-pois-PoiInfo__info--239IR")
        is_first = True
    else:
        main_page = driver.find_element_by_class_name('scrollAdMain')
        attraction_elements = main_page.find_elements_by_class_name('listing_info')
        is_first = False
    print("I found the elements!")

    attraction_names = []
    for attraction_element in attraction_elements[start:end]:      
        if is_first:
            link = attraction_element.find_element_by_tag_name('h3')
            attraction_name = link.text[link.text.find('.')+2 :]
        else:
            div_class = attraction_element.find_element_by_class_name('tracking_attraction_title')
            link = div_class.find_element_by_tag_name('a')
            attraction_name = link.text
        attraction_names.append((attraction_name, link))   
    print("I found the attraction_names")
        
    attraction_dict = {}
    for attraction_name, link in attraction_names:
        if len(attraction_dict) < end:
            stime = time.time()
            attraction_dict[attraction_name] = load_reviews(link)
            print(attraction_name,  " " + str(time.time() -stime) + "초 걸림 ", len(attraction_dict[attraction_name]))
            if len(attraction_dict) % 5 == 0:
                data_to_csv(attraction_dict)
                print("저장했어요!")
        else:
            data_to_csv(attraction_dict)
            print("저장했어요!")
            break

In [6]:
def dict_n(start, end):
    '''
    입력 : 크롤링할 즐길거리의 시작 번호와 끝 번호
    출력 : 해당 구간의 즐길거리 리뷰들csv로 출력
    '''    
    #처음 한 번만 see more을 누른다.
    is_first_page = True
    try:
        time.sleep(1)
        show = driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__see_more--2Vsb-')
        driver.execute_script("arguments[0].click();", show)
        is_first_page = True
        print("it is first page!")
    except NoSuchElementException:
        print('continue!')
        is_first_page = False
    except:
        print("exception")
    
    time.sleep(5)
    keep_going = True
    while keep_going:
        
        # 현재 페이지 번호 알아내기

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        print("I made a soup")
        if is_first_page:
            selected_page = 1
            page_numbers = driver.find_element_by_class_name('attractions-attraction-overview-main-Pagination__container--PUXGq')
            if start // 30 > 0:
                if start // 30 > 6:
                    target_num = 6
                else:
                    target_num = (start // 30) + 1
                target_page = page_numbers.find_element_by_link_text(str(target_num))
                time.sleep(5)
                driver.execute_script("arguments[0].click();", target_page)
                time.sleep(3)
                selected_page = target_num
                page_numbers = driver.find_element_by_class_name('pageNumbers')
        else:
            selected_page = int(soup.find("div", class_='pageNumbers').find(class_ = 'current').text)
            page_numbers = driver.find_element_by_class_name('pageNumbers')
        print("I find the current page")
        
        # 시작점이 뒷 페이지면 지나간다다
        if start <= selected_page * 30:
            strat_in_page = start-(selected_page-1)*30
            if strat_in_page < 0:
                strat_in_page = 0
            assert strat_in_page <= 30
            
            end_in_page = end-(selected_page-1)*30
            if end_in_page > 30:
                end_in_page = 30                
            assert 0 <= end_in_page
            load_page(strat_in_page, end_in_page)
            
            if is_first_page:
                is_first_page = False
        
        # 끝나는 지점을 지나갔으면 루프를 끝낸다.
        if (selected_page - 1) * 30 >= end:
            keep_going = False
            break
        # 다음 페이지로 넘어가기
        time.sleep(5)
        next_num = str(selected_page + 1)
        next_page = page_numbers.find_element_by_link_text(next_num)
        driver.execute_script("arguments[0].click();", next_page)

In [20]:
options = webdriver.ChromeOptions()
options.add_argument("lang=en-CA")
options.add_argument('window-size=1920x1080')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver_address = r'C:\Users\Jeong\chromedriver.exe'
driver = webdriver.Chrome(driver_address, chrome_options=options)
wait = WebDriverWait(driver, timeout=2)
driver.implicitly_wait(5)
# tripadvisor에 접근한다.
driver.get('https://www.tripadvisor.com/Attractions-g294197-Activities-Seoul.html')
#driver.get('https://www.tripadvisor.com/Attractions-g294197-Activities-oa30-Seoul.html')
#원래 있던 리스트 창을 저장해둔다
original_window = driver.current_window_handle

dict_n(start = 0,end = 10)

C:\Users\Jeong\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


it is first page!
I made a soup
I find the current page
I am in the load_page
I found the elements!
I found the attraction_names
error
error
error
error
Gyeongbokgung Palace  83.70809626579285초 걸림  5
error
error
error
error
None
Bukhansan National Park  78.1346435546875초 걸림  5
error
error
error
error
Myeongdong Shopping Street  69.01786589622498초 걸림  5
error
error
error
error
Changdeokgung Palace  68.85589742660522초 걸림  5
error
error
error
error
The War Memorial of Korea  68.5052387714386초 걸림  5
저장했어요!
error
error
error
error
National Museum of Korea  67.95286107063293초 걸림  5
error
error
error
error
Insadong  67.18243908882141초 걸림  5
error
error
error
error
Trickeye Museum Seoul  67.56653642654419초 걸림  5
error
error
error
error
None
Lotte World Tower & Mall  68.23452615737915초 걸림  5
error
error
error
error
KTX (Korea Train Express)  66.27493810653687초 걸림  5
저장했어요!
I made a soup
I find the current page


AssertionError: 